In [3]:
import pandas as pd
import numpy as np
import random
import locale


In [4]:
# Criando as datas de 01/01/2024 até 30/04/2024
datas = pd.date_range(start='2024-01-01', end='2025-04-30', freq='D')

# Criando um DataFrame com as datas
df = pd.DataFrame({'data': datas})

# Define o locale para português do Brasil
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')  # No Windows pode ser 'Portuguese_Brazil.1252'

# Gera o nome do dia da semana em português e pega os 3 primeiros caracteres
df['dia_semana'] = df['data'].dt.strftime('%A').str[:3].str.lower()

# Filtrando apenas dias úteis (segunda a sexta)
dias_uteis = df[df['dia_semana'].isin(['seg', 'ter', 'qua', 'qui', 'sex'])]

# Média de vendas nos dias úteis
amostra_vendas = np.random.randint(29, 58, size=len(dias_uteis))
media_uteis = int(np.mean(amostra_vendas))

# Função para gerar valores aleatórios de produto, oferta, valor e promo
def gera_valores_aleatorios():
    # Nome do produto
    letras = ['A', 'B', 'C', 'D']
    produto = random.choice(letras)

    # Gerando valores aleatórios 'S' ou 'N' para a coluna 'oferta'
    oferta = random.choice(['S', 'N'])

    # Mapeando os valores dos produtos
    valores = {'A': 129.90, 'B': 119.90, 'C': 109.90, 'D': 99.90}
    valor = valores[produto]

    # Lista de promoções possíveis
    promocoes = ['mensal', '30_P', '50_P']

    

    # Função para aplicar a lógica de promoção
    def atribui_promocao(oferta):
        if oferta == 'S':
            return random.choice(promocoes)
        else:
            return 'Sem_Oferta'

    # Atribuindo promoção
    promo = atribui_promocao(oferta)

    return produto, oferta, valor, promo

# Função para definir quantas vendas por dia
def num_vendas(dia):
    if dia == 'dom':
        return 0
    elif dia == 'sab':
        return max(1, media_uteis // 2)
    else:
        return random.randint(29, 57)
    

# Gerando vendas para cada linha, com valores aleatórios
df_vendas = df.loc[df.index.repeat(df['dia_semana'].apply(num_vendas))].reset_index(drop=True)

# Gerando valores aleatórios para cada linha de venda
df_vendas['produto'], df_vendas['oferta'], df_vendas['valor'], df_vendas['promo'] = zip(*[gera_valores_aleatorios() for _ in range(len(df_vendas))])

# Exibindo uma amostra das vendas geradas
print(df_vendas.head())

        data dia_semana produto oferta  valor       promo
0 2024-01-01        seg       A      S  129.9        50_P
1 2024-01-01        seg       C      N  109.9  Sem_Oferta
2 2024-01-01        seg       A      S  129.9        50_P
3 2024-01-01        seg       C      S  109.9        30_P
4 2024-01-01        seg       A      S  129.9        50_P


In [5]:
# Exibindo as primeiras linhas do DataFrame
df_vendas

,data,dia_semana,produto,oferta,valor,promo
0,2024-01-01,seg,A,S,129.9,50_P
1,2024-01-01,seg,C,N,109.9,Sem_Oferta
2,2024-01-01,seg,A,S,129.9,50_P
3,2024-01-01,seg,C,S,109.9,30_P
4,2024-01-01,seg,A,S,129.9,50_P
...,...,...,...,...,...,...
17777,2025-04-30,qua,B,N,119.9,Sem_Oferta
17778,2025-04-30,qua,C,S,109.9,mensal
17779,2025-04-30,qua,A,S,129.9,30_P
17780,2025-04-30,qua,A,N,129.9,Sem_Oferta


In [6]:
# Criando a Vigencia

df_vendas['vigencia'] = df_vendas['promo'].apply(lambda x: 12 if x == 'mensal' else (0 if x == 'Sem_Oferta' else 3))

# Exibindo o DataFrame para verificar
print(df_vendas.head())


        data dia_semana produto oferta  valor       promo  vigencia
0 2024-01-01        seg       A      S  129.9        50_P         3
1 2024-01-01        seg       C      N  109.9  Sem_Oferta         0
2 2024-01-01        seg       A      S  129.9        50_P         3
3 2024-01-01        seg       C      S  109.9        30_P         3
4 2024-01-01        seg       A      S  129.9        50_P         3


In [7]:
# Criando valor da oferta
df_vendas['oferta_vl'] = df_vendas.apply(
    lambda row: row['valor'] - 12 if row['promo'] == 'mensal' else
                row['valor'] * 0.7 if row['promo'] == '30_P' else
                row['valor'] * 0.5 if row['promo'] == '50_P' else
                row['valor'],
    axis=1
)

# Exibindo o DataFrame para verificar
df_vendas

# data |	produto	| oferta	| valor	| promo	| vigencia	| oferta_vl	| vigencia_ctt


,data,dia_semana,produto,oferta,valor,promo,vigencia,oferta_vl
0,2024-01-01,seg,A,S,129.9,50_P,3,64.95
1,2024-01-01,seg,C,N,109.9,Sem_Oferta,0,109.90
2,2024-01-01,seg,A,S,129.9,50_P,3,64.95
3,2024-01-01,seg,C,S,109.9,30_P,3,76.93
4,2024-01-01,seg,A,S,129.9,50_P,3,64.95
...,...,...,...,...,...,...,...,...
17777,2025-04-30,qua,B,N,119.9,Sem_Oferta,0,119.90
17778,2025-04-30,qua,C,S,109.9,mensal,12,97.90
17779,2025-04-30,qua,A,S,129.9,30_P,3,90.93
17780,2025-04-30,qua,A,N,129.9,Sem_Oferta,0,129.90


In [8]:
# Criando o Ticket Médio
# Calculando o ticket médio com base na vigência e no valor da oferta

df_vendas['ticket_medio'] = df_vendas.apply(
    lambda row: round((3 * row['oferta_vl'] + 9 * row['valor']) / 12, 2) if row['vigencia'] == 3 else
                round(row['oferta_vl'], 2),
    axis=1
)

In [9]:
df_vendas

,data,dia_semana,produto,oferta,valor,promo,vigencia,oferta_vl,ticket_medio
0,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66
1,2024-01-01,seg,C,N,109.9,Sem_Oferta,0,109.90,109.90
2,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66
3,2024-01-01,seg,C,S,109.9,30_P,3,76.93,101.66
4,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66
...,...,...,...,...,...,...,...,...,...
17777,2025-04-30,qua,B,N,119.9,Sem_Oferta,0,119.90,119.90
17778,2025-04-30,qua,C,S,109.9,mensal,12,97.90,97.90
17779,2025-04-30,qua,A,S,129.9,30_P,3,90.93,120.16
17780,2025-04-30,qua,A,N,129.9,Sem_Oferta,0,129.90,129.90


In [11]:
import pandas as pd
import random

# Lista de gerentes (nome e sobrenome)
gerentes = ["Carlos Souza", "Fernanda Lima", "João Pereira", "Mariana Alves", "Ricardo Rocha", "Patrícia Costa"]

# Atribuir gerente aleatoriamente para cada linha
df_vendas["Gerente"] = random.choices(gerentes, k=len(df_vendas))

# Gerar 36 nomes de vendedores
nomes = ["Ana", "Bruno", "Camila", "Daniel", "Eduarda", "Felipe", "Giovana", "Henrique", "Isabela", "José", "Karen", "Lucas",
         "Marina", "Nicolas", "Olívia", "Pedro", "Quésia", "Rafael", "Sabrina", "Tiago", "Ursula", "Victor", "Wesley", "Ximena",
         "Yuri", "Zuleica", "Amanda", "Caio", "Débora", "Elias", "Fabiana", "Gabriel", "Heitor", "Ingrid", "Jonas", "Larissa"]
sobrenomes = ["Silva", "Santos", "Oliveira", "Souza", "Lima", "Costa"]
vendedores = [f"{random.choice(nomes)} {random.choice(sobrenomes)}" for _ in range(36)]


# Mapear 6 vendedores por gerente
mapa_gerente_vendedores = {gerente: vendedores[i*6:(i+1)*6] for i, gerente in enumerate(gerentes)}

# Criar lista com todos vendedores e seus respectivos gerentes
relacao_vendedor_gerente = []
for gerente, vends in mapa_gerente_vendedores.items():
    for vendedor in vends:
        relacao_vendedor_gerente.append({"vendedor": vendedor, "Gerente": gerente})

# DataFrame com essa hierarquia
df_hierarquia = pd.DataFrame(relacao_vendedor_gerente)


# Atribuir vendedor aleatoriamente de acordo com o gerente da linha
def sortear_vendedor(gerente):
    return random.choice([v["vendedor"] for v in relacao_vendedor_gerente if v["Gerente"] == gerente])

df_vendas["vendedor"] = df_vendas["Gerente"].apply(sortear_vendedor)


# Gerar IDs únicos para os vendedores
df_hierarquia["id_vendedor"] = [f"V{str(i+1).zfill(3)}" for i in range(len(df_hierarquia))]

# Merge no df_vendas para adicionar o ID
df_vendas = df_vendas.merge(df_hierarquia, on=["vendedor", "Gerente"], how="left")

In [12]:
df_vendas

,data,dia_semana,produto,oferta,valor,promo,vigencia,oferta_vl,ticket_medio,Gerente,vendedor,id_vendedor
0,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66,João Pereira,Ana Santos,V013
1,2024-01-01,seg,C,N,109.9,Sem_Oferta,0,109.90,109.90,Ricardo Rocha,Ana Oliveira,V028
2,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66,Patrícia Costa,Bruno Oliveira,V032
3,2024-01-01,seg,C,S,109.9,30_P,3,76.93,101.66,Fernanda Lima,Lucas Costa,V007
4,2024-01-01,seg,A,S,129.9,50_P,3,64.95,113.66,João Pereira,Ximena Lima,V016
...,...,...,...,...,...,...,...,...,...,...,...,...
17777,2025-04-30,qua,B,N,119.9,Sem_Oferta,0,119.90,119.90,Carlos Souza,Larissa Oliveira,V005
17778,2025-04-30,qua,C,S,109.9,mensal,12,97.90,97.90,Carlos Souza,Zuleica Silva,V001
17779,2025-04-30,qua,A,S,129.9,30_P,3,90.93,120.16,Patrícia Costa,Amanda Silva,V031
17780,2025-04-30,qua,A,N,129.9,Sem_Oferta,0,129.90,129.90,Mariana Alves,Camila Oliveira,V021
